In [1]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization


c:\Users\prati\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\dtypes.py:29: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from tensorflow.python.lib.core import _pywrap_bfloat16
c:\Users\prati\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\dtypes.py:511: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
c:\Users\prati\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\framework\dtypes.py:563: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted 

In [2]:
ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 32

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.999

In [3]:
class DQNSolver:

    def __init__(self, state_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(16, input_shape=(
            state_space,), activation="relu"))
        #self.model.add(BatchNormalization())
        self.model.add(Dense(16, activation="relu"))
        #self.model.add(BatchNormalization())
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

        self.target_model = Sequential()
        self.target_model.add(Dense(16, input_shape=(
            state_space,), activation="relu"))
        #self.target_model.add(BatchNormalization())
        self.target_model.add(Dense(16, activation="relu"))
       # self.target_model.add(BatchNormalization())
        self.target_model.add(Dense(self.action_space, activation="linear"))
        self.target_model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        # if (len(self.memory) > 0):
        #     self.memory.pop()
        self.memory.append((state, action, reward, next_state, done))
    
    def forget(self):
        self.memory.clear()

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])
    def update_target_model(self):
        """
        Updates the current target_q_net with the q_net which brings all the
        training in the q_net to the target_q_net.
        :return: None
        """
        self.target_model.set_weights(self.model.get_weights())
    def decrement_epsilon(self, time,episodes):
        '''
        if self.epsilon > self.min_eps:
            self.epsilon *= self.decay_coe
        else:
            self.epsilon = self.min_eps
        '''
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)
        #self.exploration_rate=max((1-(1/(1+np.exp(-time/episodes)))),0.01)
        
    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            
            if not terminal:
                
                q_update = (reward + GAMMA *
                            np.amax(self.target_model.predict(state_next)[0]))
            
            q_values = self.model.predict(state)
            q_values[0][int(action)] = q_update
        self.model.fit(state, q_values, verbose=0)
            #print("4")
        

In [4]:
rewards=[]

In [5]:

observation_space = 4
action_space = 2
dqn_solver = DQNSolver(observation_space, action_space)


c:\Users\prati\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [6]:
# batch = random.sample(dqn_solver.memory, BATCH_SIZE)
        
# for state, action, reward, state_next, terminal in batch:
#     q_update = reward
#     q_values = dqn_solver.model.predict(state)
        
#     if not terminal:
#         q_update = (reward + GAMMA *
#                             np.amax(dqn_solver.target_model.predict(state_next)[0]))
#     q_values[0][int(action)] = q_update
#     dqn_solver.model.fit(state, q_values, verbose=0)

In [7]:
def cartpole(dqn_solver,runs):
    env = gym.make(ENV_NAME)
    #
    # +score_logger = ScoreLogger(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    run = 0
    x=0
    while run<runs:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])

        step = 0
        temp_reward=0
        #print("1")
        while True:
            
           # print(step)
            step += 1
            x+=1
            #env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info, = env.step(action)
            reward = reward if not terminal else -reward
            temp_reward+=reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                
                print("Run: " + str(run) + ", exploration: " +
                      str(dqn_solver.exploration_rate) + ", score: " + str(step))
                #score_logger.add_score(step, run)
                break
            dqn_solver.experience_replay()
            if((x%5==0)and(len(dqn_solver.memory)>32)):
                dqn_solver.update_target_model()
        dqn_solver.decrement_epsilon(run,runs)
            
        rewards.append(temp_reward)

In [8]:
cartpole(dqn_solver,10)

1
Run: 1, exploration: 1.0, score: 16
1
Run: 2, exploration: 0.999, score: 20
1
Run: 3, exploration: 0.998001, score: 11
1
Run: 4, exploration: 0.997002999, score: 15
1
Run: 5, exploration: 0.996005996001, score: 12
1
Run: 6, exploration: 0.995009990004999, score: 9
1
Run: 7, exploration: 0.994014980014994, score: 20
1
Run: 8, exploration: 0.993020965034979, score: 25
1
Run: 9, exploration: 0.9920279440699441, score: 12
1
Run: 10, exploration: 0.9910359161258742, score: 84


In [9]:
dqn_solver.model.weights

[<tf.Variable 'module_wrapper/dense/kernel:0' shape=(4, 16) dtype=float32, numpy=
 array([[ 0.24766822, -0.2786294 ,  0.31231207, -0.30911052,  0.38559648,
         -0.23584603,  0.4099816 , -0.36110044,  0.31454533, -0.22437836,
          0.6071396 , -0.50984806,  0.47504812, -0.06682788,  0.15601961,
         -0.45580864],
        [ 0.27539062,  0.6124408 , -0.4234629 ,  0.03728338, -0.33311036,
          0.18592076,  0.4127705 , -0.41132817, -0.32491162, -0.10505119,
          0.40656233,  0.54466975, -0.49028116, -0.2833719 , -0.20512013,
         -0.23501639],
        [-0.3439244 , -0.5185239 , -0.37040877,  0.44760242, -0.45448148,
         -0.00631185, -0.41928175, -0.31191802,  0.11143769,  0.01995319,
          0.2996801 ,  0.28088695,  0.24731809, -0.27126077,  0.2718433 ,
         -0.37017798],
        [-0.19223362, -0.11348143,  0.1928686 ,  0.570388  ,  0.44429332,
          0.30893078,  0.12665807, -0.11262645, -0.4090472 , -0.42888945,
         -0.26451647,  0.07971771, 

In [10]:
cartpole(dqn_solver,10)

1
Run: 1, exploration: 0.9900448802097482, score: 44
1
Run: 2, exploration: 0.9890548353295385, score: 23
1
Run: 3, exploration: 0.988065780494209, score: 15
1
Run: 4, exploration: 0.9870777147137147, score: 12
1
Run: 5, exploration: 0.986090636999001, score: 18
1
Run: 6, exploration: 0.9851045463620021, score: 21
1
Run: 7, exploration: 0.98411944181564, score: 18
1
Run: 8, exploration: 0.9831353223738244, score: 15
1
Run: 9, exploration: 0.9821521870514506, score: 50
1
Run: 10, exploration: 0.9811700348643991, score: 11


In [11]:
dqn_solver.model.weights

[<tf.Variable 'module_wrapper/dense/kernel:0' shape=(4, 16) dtype=float32, numpy=
 array([[ 0.2504911 , -0.27350375,  0.3482872 , -0.2917877 ,  0.39929426,
         -0.2428883 ,  0.41833884, -0.31877542,  0.3217369 , -0.21522504,
          0.61572236, -0.49948615,  0.4636316 , -0.06923992,  0.17053244,
         -0.44917986],
        [ 0.30850232,  0.63704896, -0.4641002 , -0.02179247, -0.37220675,
          0.18929943,  0.40957037, -0.42689285, -0.27723444, -0.0651797 ,
          0.43909827,  0.58155143, -0.4918555 , -0.28843123, -0.24088138,
         -0.20477523],
        [-0.35332143, -0.49919698, -0.37323895,  0.4645406 , -0.44926113,
         -0.00231136, -0.40813878, -0.3129588 ,  0.11276748,  0.03178839,
          0.3154589 ,  0.2625025 ,  0.24553682, -0.25222275,  0.2756234 ,
         -0.37994036],
        [-0.22112767, -0.13102244,  0.22473031,  0.62070817,  0.47635865,
          0.3063321 ,  0.13222119, -0.10199234, -0.44932705, -0.4623146 ,
         -0.28997672,  0.04448924, 